In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers



train_url = 'KDDTrain+.csv'
test_url = 'KDDTest+.csv'

training_df = pd.read_csv(train_url)
testing_df = pd.read_csv(test_url)


In [2]:
training_df.head()

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.25,0.26,0.27,0.05,0.28,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.0,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.0,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.0,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.0,1.00,neptune,21


In [3]:
testing_df.head()

,0,tcp,private,REJ,0.1,0.2,0.3,0.4,0.5,0.6,...,0.04.1,0.06.1,0.22,0.23,0.24,0.25,1.2,1.3,neptune,21
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,15
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,11
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,21


# -> Preprocessing

In [4]:
columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome',
    'difficulty'
]

training_df.columns = columns
testing_df.columns = columns

In [5]:
print("Training set has {} rows.".format(len(training_df)))
print("Testing set has {} rows.".format(len(testing_df)))

Training set has 125972 rows.
Testing set has 22542 rows.


In [6]:
training_outcomes=training_df["outcome"].unique()
testing_outcomes=testing_df["outcome"].unique()
print("The training set has {} possible outcomes \n".format(len(training_outcomes)) )
print(", ".join(training_outcomes)+".")
print("\nThe testing set has {} possible outcomes \n".format(len(testing_outcomes)))
print(", ".join(testing_outcomes)+".")



The training set has 23 possible outcomes 

normal, neptune, warezclient, ipsweep, portsweep, teardrop, nmap, satan, smurf, pod, back, guess_passwd, ftp_write, multihop, rootkit, buffer_overflow, imap, warezmaster, phf, land, loadmodule, spy, perl.

The testing set has 38 possible outcomes 

neptune, normal, saint, mscan, guess_passwd, smurf, apache2, satan, buffer_overflow, back, warezmaster, snmpgetattack, processtable, pod, httptunnel, nmap, ps, snmpguess, ipsweep, mailbomb, portsweep, multihop, named, sendmail, loadmodule, xterm, worm, teardrop, rootkit, xlock, perl, land, xsnoop, sqlattack, ftp_write, imap, udpstorm, phf.


In [7]:
# lists to hold our attack classifications
dos_attacks = ['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm']
probe_attacks = ['ipsweep','mscan','nmap','portsweep','saint','satan']
privilege_attacks = ['buffer_overflow','loadmdoule','perl','ps','rootkit','sqlattack','xterm']
access_attacks = ['ftp_write','guess_passwd','http_tunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xclock','xsnoop']


# Our new labels
classes = ['Normal','DoS','Probe','Privilege','Access']

#Helper function to label samples to 5 classes
def label_attack (row):
    if row["outcome"] in dos_attacks:
        return classes[1]
    if row["outcome"] in probe_attacks:
        return classes[2]
    if row["outcome"] in privilege_attacks:
        return classes[3]
    if row["outcome"] in access_attacks:
        return classes[4]
    return classes[0]


#We combine the datasets temporarily to do the labeling
test_samples_length = len(testing_df)
df=pd.concat([training_df,testing_df])
df["Class"]=df.apply(label_attack,axis=1)


# The old outcome field is dropped since it was replaced with the Class field, the difficulty field will be dropped as well.
df=df.drop("outcome",axis=1)
df=df.drop("difficulty",axis=1)

# we again split the data into training and test sets.
training_df= df.iloc[:-test_samples_length, :]
testing_df= df.iloc[-test_samples_length:,:]


In [8]:
training_outcomes=training_df["Class"].unique()
testing_outcomes=testing_df["Class"].unique()
print("The training set has {} possible outcomes \n".format(len(training_outcomes)) )
print(", ".join(training_outcomes)+".")
print("\nThe testing set has {} possible outcomes \n".format(len(testing_outcomes)))
print(", ".join(testing_outcomes)+".")

The training set has 5 possible outcomes 

Normal, DoS, Access, Probe, Privilege.

The testing set has 5 possible outcomes 

DoS, Normal, Probe, Access, Privilege.


In [9]:
training_df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,Normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,Normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,DoS
125968,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,Normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,Normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS


In [10]:
# Helper function for scaling continous values
def minmax_scale_values(training_df, testing_df, col_name):
    scaler = MinMaxScaler()
    scaler = scaler.fit(training_df[col_name].values.reshape(-1, 1))
    train_values_standardized = scaler.transform(training_df[col_name].values.reshape(-1, 1))
    training_df[col_name] = train_values_standardized
    test_values_standardized = scaler.transform(testing_df[col_name].values.reshape(-1, 1))
    testing_df[col_name] = test_values_standardized


#Helper function for one hot encoding
def encode_text(training_df,testing_df, name):
    training_set_dummies = pd.get_dummies(training_df[name])
    testing_set_dummies = pd.get_dummies(testing_df[name])
    for x in training_set_dummies.columns:
        dummy_name = "{}_{}".format(name, x)
        training_df[dummy_name] = training_set_dummies[x]
        if x in testing_set_dummies.columns :
            testing_df[dummy_name]=testing_set_dummies[x]
        else :
            testing_df[dummy_name]=np.zeros(len(testing_df))
    training_df.drop(name, axis=1, inplace=True)
    testing_df.drop(name, axis=1, inplace=True)


sympolic_columns=["protocol_type","service","flag"]
label_column="Class"
for column in df.columns :
    if column in sympolic_columns:
        encode_text(training_df,testing_df,column)
    elif not column == label_column:
        minmax_scale_values(training_df,testing_df, column)

In [11]:
training_df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.000000,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,0.000000,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,0.000000,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
125968,0.000186,7.608895e-08,1.106923e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
125969,0.000000,1.616709e-06,2.931438e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
125970,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
# Assuming training_df is already loaded
for column in training_df.columns:
    print(column)


duration
src_bytes
dst_bytes
land
wrong_fragment
urgent
hot
num_failed_logins
logged_in
num_compromised
root_shell
su_attempted
num_root
num_file_creations
num_shells
num_access_files
num_outbound_cmds
is_host_login
is_guest_login
count
srv_count
serror_rate
srv_serror_rate
rerror_rate
srv_rerror_rate
same_srv_rate
diff_srv_rate
srv_diff_host_rate
dst_host_count
dst_host_srv_count
dst_host_same_srv_rate
dst_host_diff_srv_rate
dst_host_same_src_port_rate
dst_host_srv_diff_host_rate
dst_host_serror_rate
dst_host_srv_serror_rate
dst_host_rerror_rate
dst_host_srv_rerror_rate
Class
protocol_type_icmp
protocol_type_tcp
protocol_type_udp
service_IRC
service_X11
service_Z39_50
service_aol
service_auth
service_bgp
service_courier
service_csnet_ns
service_ctf
service_daytime
service_discard
service_domain
service_domain_u
service_echo
service_eco_i
service_ecr_i
service_efs
service_exec
service_finger
service_ftp
service_ftp_data
service_gopher
service_harvest
service_hostnames
service_http
serv

# -> Training Module

In [13]:
x,y=training_df,training_df.pop("Class").values
x=x.values
x_test,y_test=testing_df,testing_df.pop("Class").values
x_test=x_test.values
y0=np.ones(len(y),np.int8)
y0[np.where(y==classes[0])]=0
y0_test=np.ones(len(y_test),np.int8)
y0_test[np.where(y_test==classes[0])]=0

In [14]:

x.shape

(125972, 122)

In [15]:
x_test.shape

(22542, 122)

In [16]:
y.shape

(125972,)

In [17]:
y_test.shape

(22542,)

# -> Build the Autoencoder model

In [18]:
def getModel():
    inp = Input(shape=(x.shape[1],))
    d1 = Dropout(0.5)(inp)
    encoded = Dense(8, activation='relu', activity_regularizer=regularizers.l2(10e-5))(d1)
    decoded = Dense(x.shape[1], activation='relu')(encoded)
    autoencoder = Model(inp, decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

autoencoder = getModel()
history = autoencoder.fit(x[np.where(y0==0)], x[np.where(y0==0)],
               epochs=10,
               batch_size=100,
               shuffle=True,
               validation_split=0.1
               )

# Print the summary of the autoencoder model
print(autoencoder.summary())

# Extract the number of features from the encoded layer
num_features = autoencoder.layers[2].output_shape[1]

print(f"Number of features in the encoded layer: {num_features}")

Epoch 1/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0187 - val_loss: 0.0070
Epoch 2/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0089 - val_loss: 0.0056
Epoch 3/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0078 - val_loss: 0.0048
Epoch 4/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0073 - val_loss: 0.0046
Epoch 5/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0071 - val_loss: 0.0044
Epoch 6/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0070 - val_loss: 0.0043
Epoch 7/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0069 - val_loss: 0.0042
Epoch 8/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0068 - val_loss: 0.0042
Epoch 9/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0067 - val_loss: 0.0040
Epoch 10/10
607/607 [==============================] - 2s 3ms/step - loss: 0.0066 - val_loss: 0.0040

In [19]:
feature_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=2).output)
features = feature_model.predict(x)

3937/3937 [==============================] - 6s 1ms/step


# -> Prediction module

# -> Machine learning model

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Assuming you have features (X) and labels (y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

# Initialize all the models with important parameters
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=5)
lr_model = LogisticRegression(max_iter=1000)
svm_model = SVC(kernel='rbf', C=1.0)
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)

# Train all the models
rf_model.fit(X_train, y_train)
knn_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
dt_model.fit(X_train, y_train)

# Predict using all the models
rf_preds = rf_model.predict(X_test)
knn_preds = knn_model.predict(X_test)
lr_preds = lr_model.predict(X_test)
svm_preds = svm_model.predict(X_test)
gb_preds = gb_model.predict(X_test)
dt_preds = dt_model.predict(X_test)

# Evaluate all the models
rf_accuracy = accuracy_score(y_test, rf_preds)
knn_accuracy = accuracy_score(y_test, knn_preds)
lr_accuracy = accuracy_score(y_test, lr_preds)
svm_accuracy = accuracy_score(y_test, svm_preds)
gb_accuracy = accuracy_score(y_test, gb_preds)
dt_accuracy = accuracy_score(y_test, dt_preds)

rf_recall = recall_score(y_test, rf_preds, average='weighted')
knn_recall = recall_score(y_test, knn_preds, average='weighted')
lr_recall = recall_score(y_test, lr_preds, average='weighted')
svm_recall = recall_score(y_test, svm_preds, average='weighted')
gb_recall = recall_score(y_test, gb_preds, average='weighted')
dt_recall = recall_score(y_test, dt_preds, average='weighted')

rf_precision = precision_score(y_test, rf_preds, average='weighted')
knn_precision = precision_score(y_test, knn_preds, average='weighted')
lr_precision = precision_score(y_test, lr_preds, average='weighted')
svm_precision = precision_score(y_test, svm_preds, average='weighted')
gb_precision = precision_score(y_test, gb_preds, average='weighted')
dt_precision = precision_score(y_test, dt_preds, average='weighted')

rf_f1 = f1_score(y_test, rf_preds, average='weighted')
knn_f1 = f1_score(y_test, knn_preds, average='weighted')
lr_f1 = f1_score(y_test, lr_preds, average='weighted')
svm_f1 = f1_score(y_test, svm_preds, average='weighted')
gb_f1 = f1_score(y_test, gb_preds, average='weighted')
dt_f1 = f1_score(y_test, dt_preds, average='weighted')


print("Random Forest-AE:")
print(f"Accuracy: {rf_accuracy}")
print(f"Recall: {rf_recall}")
print(f"Precision: {rf_precision}")
print(f"F1 Score: {rf_f1}")
print()

print("K-Nearest Neighbors-AE:")
print(f"Accuracy: {knn_accuracy}")
print(f"Recall: {knn_recall}")
print(f"Precision: {knn_precision}")
print(f"F1 Score: {knn_f1}")
print()

print("Logistic Regression-AE:")
print(f"Accuracy: {lr_accuracy}")
print(f"Recall: {lr_recall}")
print(f"Precision: {lr_precision}")
print(f"F1 Score: {lr_f1}")
print()

print("Support Vector Machine:")
print(f"Accuracy: {svm_accuracy}")
print(f"Recall: {svm_recall}")
print(f"Precision: {svm_precision}")
print(f"F1 Score: {svm_f1}")
print()

print("Gradient Boosting-AE:")
print(f"Accuracy: {gb_accuracy}")
print(f"Recall: {gb_recall}")
print(f"Precision: {gb_precision}")
print(f"F1 Score: {gb_f1}")
print()

print("Decision Tree-AE:")
print(f"Accuracy: {dt_accuracy}")
print(f"Recall: {dt_recall}")
print(f"Precision: {dt_precision}")
print(f"F1 Score: {dt_f1}")
print()


Random Forest-AE:
Accuracy: 0.9854336177813058
Recall: 0.9854336177813058
Precision: 0.9850606804396084
F1 Score: 0.985051422301616

K-Nearest Neighbors-AE:
Accuracy: 0.992101607461798
Recall: 0.992101607461798
Precision: 0.992024551026183
F1 Score: 0.9920570290008032

Logistic Regression-AE:
Accuracy: 0.9366144076205596
Recall: 0.9366144076205596
Precision: 0.9313151371296637
F1 Score: 0.933594728217648

Support Vector Machine:
Accuracy: 0.9776939869021631
Recall: 0.9776939869021631
Precision: 0.9772710292962664
F1 Score: 0.9774513281820547

Gradient Boosting-AE:
Accuracy: 0.9815836475491169
Recall: 0.9815836475491169
Precision: 0.981150297659002
F1 Score: 0.9813081578682837

Decision Tree-AE:
Accuracy: 0.9673744790633062
Recall: 0.9673744790633062
Precision: 0.9668578095354614
F1 Score: 0.9666386696608544



# ->  All Model PLOT

 📅 Accuracy, recall, precision and f1-score values 📅


In [21]:
import pandas as pd
from IPython.display import display

# Define the models and metrics
models = ['Random Forest-AE', 'K-Nearest Neighbor-AE', 'Logistic Regression-AE', 'Support Vector Machine-AE', 'Gradient Boosting-AE', 'Decision Tree-AE']
accuracy = [rf_accuracy, knn_accuracy, lr_accuracy, svm_accuracy, gb_accuracy, dt_accuracy]
recall = [rf_recall, knn_recall, lr_recall, svm_recall, gb_recall, dt_recall]
precision = [rf_precision, knn_precision, lr_precision, svm_precision, gb_precision, dt_precision]
f1_score = [rf_f1, knn_f1, lr_f1, svm_f1, gb_f1, dt_f1]

# Create a DataFrame
data = {'Model': models, 'Accuracy': accuracy, 'Recall': recall, 'Precision': precision, 'F1 Score': f1_score}
df = pd.DataFrame(data)

# Display the table with a max-height style for scrolling
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)


,Model,Accuracy,Recall,Precision,F1 Score
0,Random Forest-AE,0.985434,0.985434,0.985061,0.985051
1,K-Nearest Neighbor-AE,0.992102,0.992102,0.992025,0.992057
2,Logistic Regression-AE,0.936614,0.936614,0.931315,0.933595
3,Support Vector Machine-AE,0.977694,0.977694,0.977271,0.977451
4,Gradient Boosting-AE,0.981584,0.981584,0.981150,0.981308
5,Decision Tree-AE,0.967374,0.967374,0.966858,0.966639


In [22]:
import plotly.graph_objects as go

models = ['Random Forest-AE', 'K-Nearest Neighbors-AE', 'Logistic Regression-AE', 'Support Vector Machine-AE', 'Gradient Boosting-AE', 'Decision Tree-AE']
accuracy = [rf_accuracy, knn_accuracy, lr_accuracy, svm_accuracy, gb_accuracy, dt_accuracy]
recall = [rf_recall, knn_recall, lr_recall, svm_recall, gb_recall, dt_recall]
precision = [rf_precision, knn_precision, lr_precision, svm_precision, gb_precision, dt_precision]
f1_score = [rf_f1, knn_f1, lr_f1, svm_f1, gb_f1, dt_f1]

# Create trace for each metric
trace_accuracy = go.Bar(
    y=models,
    x=accuracy,
    name='Accuracy',
    orientation='h',
    marker=dict(color='rgba(255, 0, 0, 0.6)')
)

trace_precision = go.Bar(
    y=models,
    x=precision,
    name='Precision',
    orientation='h',
    marker=dict(color='rgba(0, 255, 0, 0.6)')
)

trace_recall = go.Bar(
    y=models,
    x=recall,
    name='Recall',
    orientation='h',
    marker=dict(color='rgba(0, 0, 255, 0.6)')
)

trace_f1 = go.Bar(
    y=models,
    x=f1_score,
    name='F1-Score',
    orientation='h',
    marker=dict(color='rgba(255, 255, 0, 0.6)')
)

# Create layout
layout = dict(barmode='group',
              yaxis=dict(title='Model'),
              xaxis=dict(title='Score'))

# Create figure
fig = go.Figure(data=[trace_accuracy, trace_precision, trace_recall, trace_f1], layout=layout)

# Show the plot
fig.show()


In [23]:
import plotly.graph_objects as go

models = ['Random Forest-AE', 'K-Nearest Neighbor-AE', 'Logistic Regression-AE', 'Support Vector Machine-AE', 'Gradient Boosting-AE', 'Decision Tree-AE']
accuracy = [rf_accuracy, knn_accuracy, lr_accuracy, svm_accuracy, gb_accuracy, dt_accuracy]
recall = [rf_recall, knn_recall, lr_recall, svm_recall, gb_recall, dt_recall]
precision = [rf_precision, knn_precision, lr_precision, svm_precision, gb_precision, dt_precision]
f1 = [rf_f1, knn_f1, lr_f1, svm_f1, gb_f1, dt_f1]

# Create a trace for each metric
metrics = ['Accuracy', 'Recall', 'Precision', 'F1-Score']
traces = []

for metric, values in zip(metrics, [accuracy, recall, precision, f1]):
    trace = go.Scatter(x=models, y=values, mode='lines+markers', name=metric, marker=dict(size=10))
    traces.append(trace)

# Create the layout
layout = go.Layout(
    title='Performance Metrics for Different Models',
    xaxis=dict(title='Model'),
    yaxis=dict(title='Score'),
    legend=dict(title='Metrics'),
)

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the plot
fig.show()


In [24]:
import plotly.express as px
models = ['Random Forest-AE', 'K-Nearest Neighbors-AE', 'Logistic Regression-AE', 'Support Vector Machine-AE', 'Gradient Boosting-AE', 'Decision Tree-AE']
accuracy = [rf_accuracy, knn_accuracy, lr_accuracy, svm_accuracy, gb_accuracy, dt_accuracy]
recall = [rf_recall, knn_recall, lr_recall, svm_recall, gb_recall, dt_recall]
precision = [rf_precision, knn_precision, lr_precision, svm_precision, gb_precision, dt_precision]
f1 = [rf_f1, knn_f1, lr_f1, svm_f1, gb_f1, dt_f1]

# Create a DataFrame
import pandas as pd
df = pd.DataFrame({'Model': models, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1})

# Melt the DataFrame for plotting
df_melted = pd.melt(df, id_vars=['Model'], var_name='Metric', value_name='Score')

# Create the plot
fig = px.bar(df_melted, x='Score', y='Model', color='Metric',
             barmode='group', orientation='h',
             color_discrete_map={'Accuracy': 'blue', 'Precision': 'green', 'Recall': 'red', 'F1-Score': 'orange'})

# Update layout for better appearance
fig.update_layout(
    title='Performance Metrics for Different Models',
    yaxis_title='Model',
    xaxis_title='Score',
    legend_title='Metrics'
)

# Show the plot
fig.show()


In [25]:
import plotly.express as px

models = ['Random Forest-AE', 'K-Nearest Neighbors-AE', 'Logistic Regression-AE', 'Support Vector Machine-AE', 'Gradient Boosting-AE', 'Decision Tree-AE']
accuracy = [rf_accuracy, knn_accuracy, lr_accuracy, svm_accuracy, gb_accuracy, dt_accuracy]
recall = [rf_recall, knn_recall, lr_recall, svm_recall, gb_recall, dt_recall]
precision = [rf_precision, knn_precision, lr_precision, svm_precision, gb_precision, dt_precision]
f1 = [rf_f1, knn_f1, lr_f1, svm_f1, gb_f1, dt_f1]

# Create a DataFrame for easy plotting
import pandas as pd
df = pd.DataFrame({'Model': models, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1})

# Melt the DataFrame for visualization
df_melted = pd.melt(df, id_vars='Model', var_name='Metric', value_name='Value')

# Create the bar plot
fig = px.bar(df_melted, x='Model', y='Value', color='Metric', barmode='group',
             labels={'Value': 'Score', 'Model': 'Model'},
             title='Performance Metrics for Different Models',
             height=600)

# Customize the layout
fig.update_layout(
    yaxis=dict(title='Score'),
    xaxis=dict(title='Model'),
    legend=dict(title='Metrics')
)

# Show the plot
fig.show()



# -> Confusion Matrix Plot


In [26]:
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np

# Define the models
models = [
    ('Random Forest-AE', rf_model, X_test, y_test),
    ('K-Nearest Neighbor-AE', knn_model, X_test, y_test),
    ('Logistic Regression-AE', lr_model, X_test, y_test),
    ('Support Vector Machine-AE', svm_model, X_test, y_test),
    ('Gradient Boosting-AE', gb_model, X_test, y_test),
    ('Decision Tree-AE', dt_model, X_test, y_test)
]

# Initialize a dictionary to store confusion matrices
all_model_cm = {}

# Define the class labels
class_labels = ['Normal', 'Probe', 'Privilege', 'Access', 'DoS']

# Generate confusion matrix for each model
for model_name, model, X, y in models:
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred, labels=class_labels)
    all_model_cm[model_name] = cm

    # Create an attractive confusion matrix plot using Plotly
    fig = ff.create_annotated_heatmap(
        z=cm,
        x=class_labels,
        y=class_labels,
        colorscale='Viridis',
        showscale=True
    )

    fig.update_layout(
        title=f'Confusion Matrix - {model_name}',
        xaxis=dict(title='Predicted', side='top'),
        yaxis=dict(title='Actual'),
        font=dict(size=12)
    )

    fig.show()


